In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

ntot = 20

def simulate_grid_protest(n0=2, n1=5, n2=5, n3=5, grid_size=15, seed=0):
    N = grid_size
    total_cells = N * N
    protesting = 0
    # Initialize the random generator with the seed from the slider
    rng = np.random.default_rng(seed)
    
    # 1. SETUP THRESHOLDS
    counts = [n0, n1, n2, n3]
    threshold_list = []
    for val, count in enumerate(counts):
        threshold_list.extend([val] * int(count))
    
    remainder = total_cells - len(threshold_list)
    if remainder > 0:
        # Assign random thresholds between 4 and 8(9) [(5)6] for the rest
        threshold_list.extend(rng.integers(4, 8, size=remainder))
    else:
        threshold_list = threshold_list[:total_cells]
        
    thresholds = np.array(threshold_list)
    # Use the seeded rng for shuffling so the layout is tied to the 'seed' slider
    rng.shuffle(thresholds)
    thresholds = thresholds.reshape((N, N))
    
    # 2. INITIAL STATE (T=0 are instigators)
    state = np.where(thresholds == 0, 1, 0)
    
    # 3. SIMULATION (Moore Neighborhood)
    for _ in range(20):
        new_state = state.copy()
        for r in range(N):
            for c in range(N):
                if state[r, c] == 0:
                    r_min, r_max = max(0, r-1), min(N, r+2)
                    c_min, c_max = max(0, c-1), min(N, c+2)
                    
                    neighbor_block = state[r_min:r_max, c_min:c_max]
                    protesting_neighbors = np.sum(neighbor_block) - state[r, c]
                    
                    if protesting_neighbors >= thresholds[r, c]:
                        new_state[r, c] = 1
        
        if np.array_equal(new_state, state):
            break
        state = new_state
        protesting = 100*int(np.sum(state))/total_cells

    # 4. VISUALIZATION
    plt.figure(figsize=(9, 7))
    plt.pcolormesh(np.flipud(state), cmap='Blues', edgecolors='white', linewidth=0.5, vmin=0, vmax=1)
    
    for r in range(N):
        for c in range(N):
            disp_r = N - 1 - r
            t_val = thresholds[r, c]
            color = "white" if state[r,c] == 1 else "black"
            plt.text(c + 0.5, disp_r + 0.5, str(t_val), 
                     ha='center', va='center', fontsize=8, color=color)
    # protesting = 100*int(np.sum(state))/total_cells
    agents0 = 100*n0/ntot**2
    plt.title(f"0-agents: {agents0:0.0f}%    \n Seed: {seed:02.0f} |   Protesting: {protesting:02.0f}%", fontsize=42)
    plt.axis('off')
    plt.tight_layout()
    # plt.show()
    plt.savefig(f'grid_{n0}_seed_{seed}.png')
    

# Dashboard


interact(simulate_grid_protest, 
         n0=IntSlider(value=2, min=0, max=ntot**2/8, description='T=0'), 
         n1=IntSlider(value=50, min=0, max=ntot**2/8, description='T=1'),
         n2=IntSlider(value=50, min=0, max=ntot**2/8, description='T=2'),
         n3=IntSlider(value=50, min=0, max=ntot**2/8, description='T=3'),
         grid_size=IntSlider(value=ntot, min=10, max=25, step=1),
         seed=0)